In [59]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from datetime import datetime
import re

%matplotlib inline

In [76]:
def parse_results(file_name):

    re_time = r'.*Total time (.*) \(ms\)'
    
    runtime = None
    with open(file_name) as f:
        lines = [line.strip() for line in f.readlines()]
        for line in lines:
            #print(line)
            if re.match(re_time, line):
                match = re.match(re_time, line)
                runtime = float(match.groups()[0])
                
    if runtime is None:
        print('could not find runtime in ', file_name)
    assert(runtime is not None)
    
    return runtime


In [84]:
def make_mobilenet_table():
    results = parse_mobilenet_results()
    

def parse_mobilenet_results():
    folder = './results/'
    
    names = ['035_96_1','035_128_1',
           '035_160_1',
            '035_192_1']
   
    times = {}

    for name in names:
        filename = folder + name + '.txt'
        time = parse_results(filename)
        print(name, time)
        times[name] = time

    return times

make_mobilenet_table()

035_96_1 214285.0
035_128_1 352801.0
035_160_1 564214.0
035_192_1 788842.0


In [3]:
def parse_old_results():
    folder = './results/'
    best_times = []
    omp1_complex_times = []
    omp1_times = []
    for i in range(1,11):
        best_file = folder + 'best_' + str(i) + '.txt'
        best_time = parse_results(best_file)
        best_times.append(best_time)

    if False:
        for i in range(1,11):   
            omp1_complex_file = folder + 'omp1_complex_' + str(i) + '.txt'
            omp1_complex_time = parse_results(omp1_complex_file)
            omp1_complex_times.append(omp1_complex_time)

        for i in range(1,11):
            omp1_file = folder + 'omp1_' + str(i) + '.txt'
            omp1_time = parse_results(omp1_file)
            omp1_times.append(omp1_time)


    def fmt_line(times, batch_size):
        times_sec = [time / 1000. for time in times]
        amortized_time_ms = np.round(np.mean(times) / batch_size, 2)

        ret = str(amortized_time_ms) + ' & '
        ret += '$' +str(np.round(np.mean(times_sec), 2)) + ' \pm ' + str(np.round(np.std(times_sec), 2)) + '$ \\tabularnewline'
        return ret

    if False:
        print('omp1_times')
        print(fmt_line(omp1_times, 1024))

        print('omp1_complex_times')
        print(fmt_line(omp1_complex_times, 2048))

    print('best_times')
    print(fmt_line(best_times, 2048))


best_times
104.95 & $214.93 \pm 7.84$ \tabularnewline


In [29]:
def make_table(best_times, omp1_times, naive_times):
    return

In [24]:
for i in range(1,11):
    omp1_file = folder + 'omp1_' + str(i) + '.txt'
    omp1_time = parse_results(omp1_file)
    omp1_times.append(omp1_time)

# TF accuracies

In [55]:
def parse_tf_accuracies(file_name):

    re_top1 = r'top1_acc (.*)'
    re_top5 = r'top5_acc (.*)'
    
    top1_acc = None
    top5_acc = None
    
    runtime = None
    with open(file_name) as f:
        lines = [line.strip() for line in f.readlines()]
        for line in lines:
            #print(line)
            if re.match(re_top1, line):
                match = re.match(re_top1, line)
                top1_acc = float(match.groups()[0])
            elif re.match(re_top5, line):
                match = re.match(re_top5, line)
                top5_acc = float(match.groups()[0])
                
    if top1_acc is None:
        print('could not find top1_acc in ', file_name)
    if top5_acc is None:
        print('could not find top5_acc in ', file_name)
    #assert(top1_acc is not None)
    #assert(top5_acc is not None)
    
    return (top1_acc, top5_acc)


In [56]:
def get_tf_accuracies(image_sizes, expansion_factors):
    folder = './results/'

    accuracies = {}

    for factor in expansion_factors:
        accuracies[factor] = {}

        for size in image_sizes:
            suffix = 'acc_' + factor + '_' + str(size) + '.txt'
            filename = folder + suffix
            top1, top5 = parse_tf_accuracies(filename)

            accuracies[factor][size] = {}
            accuracies[factor][size][1] = top1
            accuracies[factor][size][5] = top5

    print(accuracies)
    return accuracies

In [57]:
def make_tf_accuracies_table(accuracies, image_sizes, expansion_factors):
    for factor in expansion_factors:
        for size in image_sizes:
            factor_str = factor.replace('0', '0.', 1)
            model_name = factor_str + '-' + str(size)
            model_name = model_name.replace('0-','-')
            top1 = np.round(accuracies[factor][size][1], 1)
            top5 = np.round(accuracies[factor][size][5], 1)
            
            table_str = model_name + ' & ' + str(top1) \
                                   + ' & ' + str(top5) \
                                   + ' & ' + ' & \\\\ ' 
            
            
            print(table_str)

In [58]:
image_sizes = [96, 128, 160, 192, 224]
expansion_factors = ['035','050', '075']

accuracies = get_tf_accuracies(image_sizes, expansion_factors)
make_tf_accuracies_table(accuracies, image_sizes, expansion_factors)

could not find top1_acc in  ./results/acc_075_224.txt
could not find top5_acc in  ./results/acc_075_224.txt
{'035': {96: {1: 42.37, 5: 67.106}, 128: {1: 50.032, 5: 74.382}, 160: {1: 56.202, 5: 79.73}, 192: {1: 58.582, 5: 81.252}, 224: {1: 60.384, 5: 82.75}}, '050': {96: {1: 48.462, 5: 73.12}, 128: {1: 56.326, 5: 79.604}, 160: {1: 60.422, 5: 82.896}, 192: {1: 63.686, 5: 85.158}, 224: {1: 65.53, 5: 86.368}}, '075': {96: {1: 54.882, 5: 78.802}, 128: {1: 61.716, 5: 83.884}, 160: {1: 65.566, 5: 86.664}, 192: {1: 68.73, 5: 88.662}, 224: {1: None, 5: None}}}
0.35-96 & 42.4 & 67.1 &  & \\ 
0.35-128 & 50.0 & 74.4 &  & \\ 
0.35-160 & 56.2 & 79.7 &  & \\ 
0.35-192 & 58.6 & 81.3 &  & \\ 
0.35-224 & 60.4 & 82.8 &  & \\ 
0.5-96 & 48.5 & 73.1 &  & \\ 
0.5-128 & 56.3 & 79.6 &  & \\ 
0.5-160 & 60.4 & 82.9 &  & \\ 
0.5-192 & 63.7 & 85.2 &  & \\ 
0.5-224 & 65.5 & 86.4 &  & \\ 
0.75-96 & 54.9 & 78.8 &  & \\ 
0.75-128 & 61.7 & 83.9 &  & \\ 
0.75-160 & 65.6 & 86.7 &  & \\ 
0.75-192 & 68.7 & 88.7 &  & \\ 


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'